In [1]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [2]:
# Descargar recursos adicionales de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hi4i\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hi4i\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hi4i\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Preprocesamiento de texto
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re



In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('french'))

In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd

import matcher  

In [114]:
def verificar_regex_mayoria(arr, regex_pattern, umbral_mayoria):
    regex_match = np.vectorize(lambda x: bool(re.match(regex_pattern, str(x))))(arr)
    cantidad_coincidencias = np.sum(regex_match)
    return cantidad_coincidencias > (umbral_mayoria * len(arr))

In [140]:
def verificar_idioma_mayoria(arr, idioma_objetivo, umbral_mayoria):
    idiomas_detectados = np.vectorize(lambda texto: detect(texto) if isinstance(texto, str) else 'Desconocido')(arr)
    cantidad_coincidencias = np.sum(idiomas_detectados == idioma_objetivo)
    return cantidad_coincidencias > (umbral_mayoria * len(arr))

In [148]:
for idx in casa2.index:

    texto = casa2[idx]

    if  verificar_regex_mayoria(texto, r'^[a-zA-Z]+$', 0.1):
      nombres_empresas_sin_nan = texto[~pd.isnull(texto)]
      print(texto)

      idiomas_empresas = np.array([detect(nombre) for nombre in nombres_empresas_sin_nan])
      print(idiomas_empresas)


      # print(detect_idioma)


['SCT12866' nan 'A6UJ9A0000DV' 'SCT13319']
['id' 'hu' 'id']
['STO183165' nan 'STO183109' 'STO181556 ']
['pt' 'pt' 'pt']
['BESTHEIM' 'BORIE MANOUX' 'Bouchard Ainé et Fils' 'DOURTHE'
 'LA PASSION DES TERROIRS' 'LES VINS DE CRUS' 'NICOLAS FEUILLATE' 'SBA'
 'VIDAL FLEURY']
['de' 'vi' 'fr' 'de' 'de' 'de' 'de' 'de' 'ca']
['BESTHEIM' 'BM' 'Bouchard Ainé et Fils' 'CVBG' 'LPDT' 'LVDC'
 'NICOLAS FEUILLATE' 'SBA' 'Vidal-Fleury']
['de' 'de' 'fr' 'en' 'fr' 'fr' 'de' 'de' 'fr']
['BEV_DET' nan]
['de']
['_det2' nan]
['da']
['Reprise' nan 'Reprise avec changement de lot + millésime'
 'Reprise avec changement de lot' 'Visuel '
 'Reprise avec changement de millésime']
['ro' 'fr' 'fr' 'es' 'fr']
['Oui' 'oui']
['fr' 'fr']
['Alsace_Lorraine' 'Bordeaux' 'Bourgogne' 'Champagne' 'Rhône']
['fr' 'fr' 'fr' 'fr' 'vi']
['Alsace' "Crémant 'Alsace" 'Alsace grand cru Mambourg' 'Pauillac'
 'Saint-Estèphe'
 'Coteaux Bourguignons ou Bourgogne grand ordinaire ou Bourgogne ordinaire'
 'Rully' 'Bourgogne Hautes Côtes de Bea

In [132]:
detect_idioma = np.vectorize(lambda texto: detect(texto) if isinstance(texto, str) else 'Desconocido')

In [43]:
df['Idioma'] = df['Texto'].apply(lambda texto: detect(texto) if isinstance(texto, str) else 'Desconocido')


pandas.core.series.Series